<a href="https://colab.research.google.com/github/yenyyo/R1-Research_internship_1/blob/main/RI_1_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#elimino una carpeta de contenido por defecto
!rm -r /content/sample_data/*
!rm -r /content/sample_data

#creo la carpeta que va a contener los logs de salida
!mkdir /content/log_aux/

#copy the .zip file with everything and unzip it
!cp -r /content/drive/MyDrive/'03 - Master'/05-RI_1/RI_1_v0.tar.gz /content/
!tar -zxvf /content/RI_1_v0.tar.gz -C /content/ >log_aux/log_unzip_RI.txt


#there is probably a waaaay more elegant way of doing this (but it works so meh)
!cp -r /content/'00-Docu'/'01-Setup'/Makefile /content/
!cp -r /content/'00-Docu'/'01-Setup'/requirements.txt /content/

#install dependencies
!make  > log_aux/log_make.txt

#delete residuals
!rm -r Makefile
!rm -r requirements.txt
!rm -r RI_1_v0.tar.gz

#bring out aux scripts
!cp /content/'01-Code'/aux_graphs.py /content/



rm: cannot remove '/content/sample_data/*': No such file or directory
rm: cannot remove '/content/sample_data': No such file or directory
mkdir: cannot create directory ‘/content/log_aux/’: File exists


In [ ]:
#installing faiss is a pain, if done by pip it dosent do it correctly (we need gpu)
!pip install -q condacolab
import condacolab
import subprocess
condacolab.install()
# !conda install -c pytorch -c nvidia faiss-gpu=1.8.0 >log_aux/log_faiss.txt
!conda install faiss-gpu cudatoolkit=10.0 -c pytorch

import cugraph
import cudf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import faiss
import anndata as ad


#my own imports


import aux_graphs as graphs
import seaborn as sns  # Import Seaborn
import pandas as pd

print(cugraph.__version__)

K_NN = 100


✨🍰✨ Everything looks OK!


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c conda-forge conda


24.02.00


In [ ]:
counts = ad.read_h5ad("/content/01-Code/data/counts_absence_presence_only.h5ad")


adata = ad.AnnData(counts)


# show data
display(adata.X)
display(adata.obs)
display(adata.var)

<1413x198944 sparse matrix of type '<class 'numpy.float64'>'
	with 18722800 stored elements in Compressed Sparse Row format>

,Dataset,subject_id,Virus Diseases,Age,Biological Sex,Racial Group,Tissue Source,repertoire_size
sample_name,,,,,,,,
860011283_TCRB,COVID-19-HUniv12Oct,748180,COVID-19 Positive,86 Years,Female,Caucasian,Blood,129743.0
INCOV053-AC-3_TCRB,COVID-19-ISB,053,COVID-19 Positive,86 Years,Male,Caucasian,Blood,86436.0
KH20-09670_TCRB,COVID-19-DLS,550040039,COVID-19 Positive,84 Years,Male,Asian or Pacific Islander,Blood,45859.0
BS-EQ-23-T1-replacement_TCRB,COVID-19-NIH/NIAID,0000142,COVID-19 Positive,49 Years,Male,Caucasian,"Blood,gDNA",57115.0
BS-HS-157_TCRB,COVID-19-NIH/NIAID,NaN,NaN,NaN,NaN,NaN,"Blood,gDNA",23186.0
...,...,...,...,...,...,...,...,...
BS-GIGI_36-replacement_TCRB,COVID-19-NIH/NIAID,0000471,COVID-19 Positive,70 Years,Female,Caucasian,"Blood,gDNA",52262.0
1328-CM-933_TCRB,COVID-19-NIH/NIAID,NaN,NaN,NaN,NaN,NaN,"Buffy Coat,gDNA",106779.0
1566746BW_TCRB,COVID-19-BWNW,1566746,COVID-19 Positive,21 Years,Female,Native American or Alaska Native,Blood,330053.0


,v_call,junction_aa
0,TRBV10-1*01,CASRDSLNTEAFF
1,TRBV10-1*01,CASRDSMNTEAFF
2,TRBV10-1*01,CASRENTGELFF
3,TRBV10-1*01,CASRGQGNTEAFF
4,TRBV10-1*01,CASRGTGELFF
...,...,...
198939,TRBV9*01,CASSVYSGNTIYF
198940,TRBV9*01,CASSVYSTDTQYF
198941,TRBV9*01,CASSVYTDTQYF
198942,TRBV9*01,CASSVYTGELFF


In [ ]:
gpu_resource = faiss.StandardGpuResources()

AttributeError: module 'faiss' has no attribute 'StandardGpuResources'

In [ ]:
X = counts.X.T.todense()
X = np.ascontiguousarray(X).astype(np.float32)

# X = X/X.sum(axis=0)  # scale the features: fraction instead of counts. Need to experiment with this, does this have an impact? (doesnt really seem to improve things)
# X = normalize(X, norm="l1", axis=0) # test: alternative way of scaling features through L2
# X = X-X.mean(axis=0) # Center the columns to scale features?

X = X-X.mean(axis=1)[:,None]  # Center the rows. Use for adjusted cosine = approx pearson correlation.
faiss.normalize_L2(X) # L2 normalize counts so that they sum up to one so that cosine similarity is measured (even though we use a dot product index)

In [ ]:
# construct index
idx = faiss.GpuIndexFlatIP(gpu_resource, X.shape[1]) # Inner product index
idx.add(X)

AttributeError: module 'faiss' has no attribute 'GpuIndexFlatIP'

In [ ]:
!pip install faiss-gpu